# Querying SQL databases

Your task is to analyze game and player statistics from the NBA 2008-2009 season provided as a relational database of play-by-play data.

You cannot use jupyter to execute the queries. To write and test the queries, use the web interface at http://pollux.vo.elte.hu/basketball/. Copy queries into the worksheet for submission.

## Excercise 1 - Simple queries

a) List all teams playing in the league during the season

b) List teams in alphabetic order but division by division

c) Count the number of persons/players in the league

**Answers**


**a) select home_team_id from game 
    union
   select away_team_id from game**


**b)
select home_team_id from game
union 
select away_team_id from game
order by 1**


**c)select count(id) from player**

## Excercise 2 - Simple joins

a) List teams by division, including the division name

b) List players by team including the uniform number

**Answers**

**a)select  d.name division_name, t.name team_name from team t , division d
where t.division_id = d.id
order by 1,2**

**b) select person.name, player.number from person
inner join player on player.person_id = person.id**


## Excercise 3 - Simple aggregation

a) Count number of persons playing for a given team

b) List all teams along with the number of players

c) Count the number of teams a given person played for

d) List all players along with the number of teams they played for

e) List players who played for more than two teams during the season

**Answers**

**a)select count(id) cnt, team_id from player
group by(team_id)
order by 1 desc**


**b)select team.name, player.person_id from team
inner join player on player.team_id = team.id**


**c)select b.name, count(team_id)
from dbo.player as a
left join dbo.person as b
on a.person_id=b.id
group by b.name**



**d)select b.name, c.name
from dbo.player as a
left join dbo.person as b
on a.person_id=b.id
left join dbo.team as c
on a.team_id=c.id**



**e)
 select b.name, count(team_id)
from dbo.player as a
left join dbo.person as b
on a.person_id=b.id
group by b.name
having count(team_id)>1**

## Excercise 4 - More complex queries with set operations and common table expressions

a) Pick a game and list all field shots and free throw attempts, including points

b) Combine the previous two into a single table and only show successful attempts

c) Take previous query and sum all points by team so that end result is available

d) Remove filter on game and get end result of each game during season

e) Modify the previous query to return a single line per game, with home team and away team points

**Answers**

**a)select e.name, d.name ,c.description from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
where a.id=1 and c.description in('3pt', 'hook bank', 'layup')**

**b)select e.name, d.name ,c.description, points from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
left join point as f
on b.cntr=f.cntr and a.id=f.game_id
where a.id=1 and c.description in('3pt', 'hook bank', 'layup')**

**c)select e.name, sum(points) point from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
left join point as f
on b.cntr=f.cntr and a.id=f.game_id
where a.id=1 and c.description in('3pt', 'hook bank', 'layup')
group by e.name**

**d)select a.id ,e.name, sum(points) points from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
left join point as f
on b.cntr=f.cntr and a.id=f.game_id
where c.description in('3pt', 'hook bank', 'layup')
group by a.id, e.name
order by 1**

**e)with temp01 as
(
select a.id, e.name, sum(points) task, g.type from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
left join point as f
on b.cntr=f.cntr and a.id=f.game_id
left join (select game_id, home_player_id, 'Home' type from jumpball
union
select game_id, away_player_id, 'Away' from jumpball) as g
on a.id=g.game_id and d.id=g.home_player_id
group by a.id, e.name, g.type
),
temp02 as
(
select a.id, a.name, sum(task) task, b.type from temp01 as a
left join (select id, name, type from temp01) as b
on a.id=b.id and a.name=b.name
group by a.id, a.name, b.type
)
select * from temp02
where type is not null
order by 1**

## Excercise 5 - a rather complex example

a)  Print season results by division. For each team, calculate with a **single_ query**

- number of games played
- wins
- losts
- winning percentage

Compare with https://en.wikipedia.org/wiki/2008–09_NBA_season#By_division

**Answer**

select *, cast ((win/sum1) as float) perent from (

select *, (win+lose) sum1 from (

select team, div, sum(lose) lose, sum(win) win from (
select team, div, case when result = 'L' then 1 end lose
,case when result = 'W' then 1 end win
from(
select b.id, b.team, b.div, case when rown = 1 then 'L' else 'W' end result 
from(
select a.*, row_number() over (partition by a.id order by a.task) rown 
From(
select a.id, e.name team, g.name div, sum(points) task from dbo.game as a
left join dbo.shot as b
on a.id=b.game_id
left join dbo.shot_type as c
on b.type=c.id
left join dbo.person as d
on b.player_id=d.id
left join dbo.team as e
on b.team_id=e.id
left join point as f
on b.cntr=f.cntr and a.id=f.game_id
left join dbo.division as g
on e.division_id=g.id
group by a.id, e.name, g.name) as a) as b) as c) as d
group by team, div) as e) as f